In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from transformers import AutoTokenizer, LongformerTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import torch
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
from hanspell import spell_checker
import re

# Train

In [2]:
data_base = "/home/aicontest/aicon_2025/data/original"

In [3]:
train_df = pd.read_csv(f"{data_base}/train.csv")
train_df.head()

,title,full_text,generated
0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0
1,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0
2,엘자스-로트링겐 평의회 공화국,엘자스-로트링겐 평의회 공화국은 1차대전 말기 독일 혁명 와중에 엘자스-로트링겐에서...,0
3,윌리엄 페니 브룩스,"윌리엄 페니 브룩스(, 1809년 8월 13일 ~ 1895년 12월 11일)는 잉글...",0
4,미그로,"미그로 또는 미그로스(""Migros"")는 스위스 최대 소매 회사이자, 최대 슈퍼마켓...",0


In [4]:
cheaker = train_df["full_text"]
print(cheaker[0])

카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5km2, 높이는 452m이다. 하와이 제도에서 가장 작은 화산섬이자 무인도이며 길이는 18km, 너비는 10km이다. 
 마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 위치한다. 고도가 낮고 북동쪽에서 불어오는 무역풍을 통해 산악 지대에서 내리는 비를 형성하지 못하기 때문에 건조한 기후를 띤다. 마우이섬 화산의 비그늘에 속해 있기 때문에 섬 전체 면적의 1/4 이상이 부식되어 있다. 
 1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬 안에는 주민들이 돌로 만든 제단에서 종교 의식을 거행한 흔적들, 주민들이 암석이나 평평한 돌에 그림을 그린 흔적들이 남아 있다. 1778년부터 1800년대까지 이 지역을 지나 항해하던 사람들의 보고에 따르면 카호올라웨섬은 무인도였고 나무도 물도 없는 불모지였다고 한다. 
 1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 사용되었지만 1853년에 폐지되었다. 1858년에는 하와이 정부가 목장 사업가들에게 카호올라웨섬을 양도했지만 가뭄과 과도한 방목으로 인해 땅이 말라갔다. 또한 강한 무역풍으로 인해 표토의 대부분이 날아가면서 붉은 경반층만 남게 되었다. 
 1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을 천연보호구역으로 지정했지만 큰 성과를 거두지 못했다. 
 1941년 12월 7일에 일어난 일본 제국 해군의 진주만 공격을 계기로 카호올라웨섬은 태평양 전쟁에 참전한 미국 병사들의 훈련소로 사용되었다. 1981년 3월 18일에는 미국 국립사적지에 등재되었다.


In [5]:
label_0 = [i for _, i in train_df.iterrows() if i['generated'] == 0]
label_1 = [i for _, i in train_df.iterrows() if i['generated'] == 1]

In [6]:
print(f"데이터 개수 : {len(train_df)}")
avg_text = sum([len(i) for i in train_df["full_text"]])/len(train_df)
print(f"평균 text 길이 : {avg_text}")
max_length = max([len(i) for i in train_df["full_text"]])
min_length = min([len(i) for i in train_df["full_text"]])
print(f"가장 길이가 긴거 : {max_length}")
print(f"가장 길이가 짧은거 : {min_length}")

데이터 개수 : 97172
평균 text 길이 : 2323.227822829622
가장 길이가 긴거 : 98549
가장 길이가 짧은거 : 393


In [22]:
count = Counter(train_df["title"]).most_common(5)
count

# 각각 하나의 title 을 가지고 있다

[('카호올라웨섬', 1),
 ('청색거성', 1),
 ('엘자스-로트링겐 평의회 공화국', 1),
 ('윌리엄 페니 브룩스', 1),
 ('미그로', 1)]

In [23]:
print(f"클래스 비율 : {sum(train_df['generated'])/len(train_df)}")
print(f"가중치 : {len(train_df)/sum(train_df['generated'])}")

클래스 비율 : 0.08227678755196971
가중치 : 12.154096310193871


In [45]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

cut = 0
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    text = row['title'] + " " + row['full_text']
    output = tokenizer(
        text,
        truncation=False,            # ✅ 자르지 않음
        padding=False,
        return_tensors="pt"
    )
    if output['input_ids'].shape[1] > 4096:
        cut += 1

print(f"잘린다고 예상되는 샘플 수: {cut} / 전체 {len(train_df)}개 중 {cut / len(train_df) * 100:.2f}%")


100%|██████████| 97172/97172 [04:57<00:00, 326.08it/s]

잘린다고 예상되는 샘플 수: 29704 / 전체 97172개 중 30.57%


# TEST

In [7]:
test_df = pd.read_csv(f"{data_base}/test.csv")
test_df.head()

,ID,title,paragraph_index,paragraph_text
0,TEST_0000,공중 도덕의 의의와 필요성,0,도덕이란 원래 개인의 자각에서 출발해 자기 의지로써 행동하는 일이다. 그러므로 도덕...
1,TEST_0001,공중 도덕의 의의와 필요성,1,도덕은 단순히 개인의 문제나 사회의 문제로 한정될 수 없다. 개인적인 측면과 사회적...
2,TEST_0002,공중 도덕의 의의와 필요성,2,"여기에 이른바 공중도덕은 실천적, 사회적 도덕의 한 부문이다. 즉, 공중 도덕이라 ..."
3,TEST_0003,공중 도덕의 의의와 필요성,3,우리가 공동 생활을 하는 데 있어서 공중 도덕이 필요함은 위에서 말한 것처럼 알 수...
4,TEST_0004,풍습과 그 개선,0,인간 사회에서는 다 함께 지켜야 할 어떤 기준이 있어 이를 따르면 옳다고 하고 따르...


In [8]:
cheaker = test_df["paragraph_text"]
print(cheaker[0])

도덕이란 원래 개인의 자각에서 출발해 자기 의지로써 행동하는 일이다. 그러므로 도덕은 어디까지나 정신의 문제이고, 각자의 마음씨에 달려있는 일이다. 여기에서 도덕의 문제는 철학적 이론으로 발전하였으며, 고상하고 심원한 이론 체계에 기울어지는 경향이 많았다. 이러한 경향으로 인해 도덕은 학식이 높은 특수한 사람만이 닦을 수 있는 것으로 여겨지며, 일반 사람은 도저히 지킬 수 없는 것처럼 오해받는 경우가 많았다. 그러나 도덕의 본질은 결코 이론에 있는 것이 아니라 실천에 있다. 이론이 필요하다면 그것은 오직 실천을 위한 도구로서만 의미를 가진다. 아무리 고상한 이론이라도 실천이 없다면 그 이론은 단순한 관념의 유희에 불과하다.


In [9]:
print(f"데이터 개수 : {len(test_df)}")

데이터 개수 : 1962


In [6]:
count = Counter(test_df["title"]).most_common()

In [ ]:
for tt, count in count:
    correct = [i for i in train_df["title"] if i == tt]
    print(f"{tt} : {len(correct)}")

# title 겹치는건 하나도 없다 --> cosine embedding similiar 로 접근해야 하네 ( RAG )

보이지 않는 재산 : 0
과거의 세상에 살고 있는 너에게 : 0
교육계의 미래를 위한 제언 : 0
‘그럼에도 불구하고’ 지켜야 할 가치 : 0
화이트칼라로봇의 등장과 메타버스에서 일하는시대 : 0
메타버스 시대가 온다 : 0
내가 만드는 모두의 권리 : 0
책을 읽고 싶어요 : 0
4차 산업혁명과 향후 한국교육의 방향 : 0
교원평가제도와 성과급: 취지와 개선점 : 0
인구감소시대 소멸위험 지자체의 생존전략 : 0
과학기술 인재로 쌓아 올릴 한국의 실력 : 0
코로나바이러스·중국·기후변화' 3C의 위기를 넘어서 : 0
개헌, 더이상 미룰 수 없는 시대적 과제 : 0
나누면 더 잘사는 국가균형발전, 더 이상 미룰 수 없는 미래다 : 0
누가 태화관길을 없앴나 : 0
국제기구가 평가한 우리나라 에너지부문 성적표 : 0
대전환기의 분기점으로서 2021년의 국가전략 : 0
청년이 살아야 대한민국이 산다 : 0
0.10% : 0
저작권! 내가 먼저 지켜야지 : 0
답은 늘 규제가 아니라 시장이었다. 앞으로도 그럴 것이다. : 0
탄소중립 선언과 미래세대를 위한 준비 : 0
건강한 잎이 많은 사회와 우리 미래의 비례 공식 : 0
나는 저작권 스타일 : 0
글로벌 블록체인 정책 협의체 출범, '미래를 향한 용기 있는 진일보' : 0
포스트 코로나19 세상에서는 새로운 인간성이 나타날 것이다 : 0
과학기술로 실력과 품격을 : 0
여가부가 필요 없는 성평등 대한민국을 꿈꾸며 : 0
미래 청사진은 아이 울음소리 울려 퍼지는 대한민국에서 : 0
이야기꾼과심술할아버지 : 0
미래의 민주주의와 민주주의 미래 : 0
감염병 확산 下의 아주 가까운 미래 : 0
코로나 대응 정보기술 활용법 : 0
코로나가 가르쳐 준 미래 생존법 '성찰과 공존' : 0
COVID-19 시대, 우리나라 국가혁신체제 어떻게 변화할 것인가? : 0
미래 정책시스템의 틀: 격변과학기술에 대한 신중한 경계 : 0
탄소국경조정의 현황과 미래영향에 관한 소고 : 0
포스트 코로나 시기를 위한 산업정책 제언 : 0
중

## 문체 혼용 비교

In [25]:
import re

def split_sentences(text):
    return re.split(r'(?<=[.?!])\s+', text.strip())

def classify_style(sentence):
    sentence = sentence.strip()

    if re.search(r'니다\.$', sentence):
        return "습니다체"
    elif re.search(r'다\.$', sentence):
        return "이다체"
    else:
        return "기타"

def analyze_paragraph_style(paragraph):
    sentences = split_sentences(paragraph)
    labeled = [(s, classify_style(s)) for s in sentences if s]

    # '기타' 제외하고 혼용 여부 판단
    styles_used = {style for _, style in labeled if style in {"습니다체", "이다체"}}
    mixed = len(styles_used) > 1

    return mixed, labeled


In [27]:
label_0_mix = []

for i in tqdm(label_0, total=len(label_0)):
    paragraph = i['full_text']
    mixed, labeled = analyze_paragraph_style(paragraph)
    label_0_mix.append(mixed)

print(f"혼용 개수 {sum(label_0_mix)} 비율 : {sum(label_0_mix)/len(label_0_mix)}")


  0%|          | 0/89177 [00:00<?, ?it/s]

100%|██████████| 89177/89177 [00:03<00:00, 22814.37it/s]

혼용 개수 7366 비율 : 0.08259977348419435


In [28]:
label_1_mix = []

for i in tqdm(label_1, total=len(label_1)):
    paragraph = i['full_text']
    mixed, labeled = analyze_paragraph_style(paragraph)
    label_1_mix.append(mixed)

print(f"혼용 개수 {sum(label_1_mix)} 비율 : {sum(label_1_mix)/len(label_1_mix)}")


100%|██████████| 7995/7995 [00:00<00:00, 22484.39it/s]

혼용 개수 5489 비율 : 0.6865540963101938


In [33]:
### test 데이터 합치기

test_csv = test_df.rename(columns={'paragraph_text': 'full_text'})
# 그룹핑하고 문단 순서대로 이어 붙이기
grouped = (
    test_csv
    .sort_values(by=["title", "paragraph_index"])
    .groupby("title")
    .agg({
        "full_text": lambda x: "\n\n".join(x),
        "paragraph_index": "count"  # 문단 개수 세기
    })
    .reset_index()
)

# 컬럼 이름 변경
grouped = grouped.rename(columns={
    "paragraph_index": "paragraph_count"
})

grouped.head()

,title,full_text,paragraph_count
0,"""루비콘 강을 건너다""",루비콘 강은 이탈리아 북부에 위치한 작은 강으로 되돌아 갈 수 없는 상황에 처해 있...,6
1,0.10%,"토렌트 다운 중 0.3% 밤 11시. 늦은 밤, 집에 돌아온 나는 자연스럽게 컴퓨터...",16
2,10년안에 세계 1등 국가 만들자,금년이 3.1운동 100주년입니다. 이 뜻깊은 해를 맞아 특별히 강조하고 싶은 점이...,10
3,2022년 1월 미래 시나리오,"2020년 1월 코로나가 확산되기 시작했을 때, 무려 1년반이 넘도록 지금 이 시간...",8
4,4차 산업혁명 시대의 국토,과거 보통 사람들의 희망은 살 집과 농사지을 땅을 마련하는 것이었고 국토가 그 희망...,9


In [36]:
label_test_mix = []

test = [i for _,i in grouped.iterrows()]

for i in tqdm(test, total=len(test)):
    paragraph = i['full_text']
    mixed, labeled = analyze_paragraph_style(paragraph)
    label_test_mix.append(mixed)

print(f"혼용 개수 {sum(label_test_mix)} 비율 : {sum(label_test_mix)/len(label_test_mix)}")


100%|██████████| 253/253 [00:00<00:00, 17110.50it/s]

혼용 개수 82 비율 : 0.3241106719367589


In [12]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

cut = 0
test_df = test_df.rename(columns={'paragraph_text': 'full_text'})
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    text = row['full_text']
    output = tokenizer(
        text,
        truncation=False,            # ✅ 자르지 않음
        padding=False,
        return_tensors="pt"
    )
    if output['input_ids'].shape[1] > 512:
        cut += 1

print(f"잘린다고 예상되는 샘플 수: {cut} / 전체 {len(test_df)}개 중 {cut / len(test_df) * 100:.2f}%")


100%|██████████| 1962/1962 [00:00<00:00, 5248.90it/s]

잘린다고 예상되는 샘플 수: 3 / 전체 1962개 중 0.15%


In [41]:
# 2. 문서(title) 단위로 문단 합치기
grouped = test_df.groupby("title")["paragraph_text"].apply(lambda x: " ".join(x)).reset_index()

# 3. 각 문서에 대해 혼용 여부 판단
grouped["is_mixed"] = grouped["paragraph_text"].apply(lambda x: int(analyze_paragraph_style(x)[0]))


# 4. 다시 원래 데이터에 혼용 여부 병합 (모든 문단에 동일 라벨)
df = test_df.merge(grouped[["title", "is_mixed"]], on="title", how="left")


sample_submission = pd.read_csv(f"{data_base}/sample_submission.csv", encoding='utf-8-sig')
all_AI = [i['is_mixed'] for _,i in df.iterrows()]
sample_submission['generated'] = all_AI

sample_submission.to_csv(f"{data_base}/문체분석.csv", index=False)


In [43]:
df = pd.read_csv(f"{data_base}/문체분석.csv")

df_0 = [i for _, i in df.iterrows() if i['generated'] == 0]
df_1 = [i for _, i in df.iterrows() if i['generated'] == 1]

print(f"0 개수 {len(df_0)} 1 개수 {len(df_1)}")

0 개수 1109 1 개수 853


In [57]:
label_0
label_1
test = test_df["paragraph_text"]

print(label_0[0]["full_text"])

카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5km2, 높이는 452m이다. 하와이 제도에서 가장 작은 화산섬이자 무인도이며 길이는 18km, 너비는 10km이다. 
 마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 위치한다. 고도가 낮고 북동쪽에서 불어오는 무역풍을 통해 산악 지대에서 내리는 비를 형성하지 못하기 때문에 건조한 기후를 띤다. 마우이섬 화산의 비그늘에 속해 있기 때문에 섬 전체 면적의 1/4 이상이 부식되어 있다. 
 1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬 안에는 주민들이 돌로 만든 제단에서 종교 의식을 거행한 흔적들, 주민들이 암석이나 평평한 돌에 그림을 그린 흔적들이 남아 있다. 1778년부터 1800년대까지 이 지역을 지나 항해하던 사람들의 보고에 따르면 카호올라웨섬은 무인도였고 나무도 물도 없는 불모지였다고 한다. 
 1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 사용되었지만 1853년에 폐지되었다. 1858년에는 하와이 정부가 목장 사업가들에게 카호올라웨섬을 양도했지만 가뭄과 과도한 방목으로 인해 땅이 말라갔다. 또한 강한 무역풍으로 인해 표토의 대부분이 날아가면서 붉은 경반층만 남게 되었다. 
 1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을 천연보호구역으로 지정했지만 큰 성과를 거두지 못했다. 
 1941년 12월 7일에 일어난 일본 제국 해군의 진주만 공격을 계기로 카호올라웨섬은 태평양 전쟁에 참전한 미국 병사들의 훈련소로 사용되었다. 1981년 3월 18일에는 미국 국립사적지에 등재되었다.


In [58]:
print(label_1[0]["full_text"])

수난곡(受難曲)은 배우의 연기 없이 무대에 올려지는 성악을 주로 한 종합 예술이다. 이러한 의미에서 오라토리오와 유사하지만, 성경의 사복음서를 기반으로 한 예수 그리스도의 생애를 주로 다루고 있다는 점에서 차이가 있다. 또한 이는 주로 독일 계열 작곡가들에게 쓰인 개념이다. 수난 또는 수난곡을 뜻하는 영어 'Passion'은 2세기에 나타난 라틴어 "passio"에서 유래하며, 예수의 생애와 고난이란 의미를 담고 있다.
사복음서에 기록된 예수님의 수난 이야기는 마태오, 마르코, 루카, 요한 복음서에 각각 담겨 있습니다. 이를 바탕으로 '마태오 수난곡', '마르코 수난곡', '루카 수난곡', '요한 수난곡'이라는 4개의 작품이 만들어졌죠. 이들 작품은 예수님의 고난과 십자가 처형을 다루고 있습니다.
옛날부터 성(聖) 금요일이나 성주간(聖週間)에는 수난극이나 이와 비슷한 행사를 하였다. 12세기경부터 복음서에 따라 그리스도 수난의 이야기를 3인의 신부가, 한 사람은 복음사가(福音史家)의 역(테너)을, 또 한 사람은 그리스도의 역(베이스)을, 나머지 한 사람은 군중의 역(알토)을 맡아 낭독조로 노래하는 습관이 되었다. 이것이 그 뒤의 수난곡의 기원이다. 이와 같은 형식으로 된 수난곡을 '코랄 수난곡'이라 하며, 대략 17세기경까지 만들어졌다. 특히 유명한 작품으로는 쉬츠의 《마태오 수난곡》이 있다. 이와 함께 16세기-17세기에는 다른 타입의 '모테토 수난곡'이 생겼다. 이것은 텍스트의 전체를 등장인물의 수와 관계없이 일관하여 모테토풍의 다성부 합창으로 노래한다. 음악사적으로 가장 중요한 것은 17세기 중엽경에 성립한 '오라토리오 수난곡'이다. 이것은 일반적으로 성서의 텍스트를 자유롭게 시로 만들어 코랄 또는 솔로의 아리아 형식으로 삽입되어 있기 때문에, 형식적으로도 아리아, 레치타티보, 합창, 통주저음, 기악반주를 썼으며, 오라토리오의 형태와 흡사하다. 이 장르의 대표적 작품으로는 바흐의 《요한 수난곡》과 고금 최대의 걸작이라고 칭찬받고 있는 《마태오 수난곡》이 

In [61]:
print(test[9])

나아가 우리가 알아두어야 할 것은 가정 생활이란 자기 한 가정만으로써 행복을 누리는 것이 아니라, 다른 가정과 더불어 국가와 긴밀한 관계에서 이루어진다는 점이다. 사람이란 원래 사회적 동물로서 개인이 홀로 살아갈 수 없기 때문이다. 행복한 가정은 국가발전에 기초가 되며, 건전한 가정은 국가의 보호를 받는다. 오늘날 자녀들은 부모의 자녀로서 가정에 국한된 사람이 아니라, 나라의 자녀들 이라는 것을 알아야한다. 국가는 가정 내부에 대해 공연한 간섭을 하지 않고 가정의 자립을 보장하지만, 가정 안에서 인권을 훼손하는 일이 있을 때는 국가에 의해 제재를 받을 수도 있는 것이다.


# 확률 보정

In [13]:
df = pd.read_csv("/home/aicontest/aicon_2025/data/original/roberta_submission_0.89131.csv")

df['generated'] = np.where(df['generated'] >= 0.99, 1,
                  np.where(df['generated'] <= 0.01, 0, df['generated']))

df.to_csv("/home/aicontest/aicon_2025/data/original/roberta_submission_0.89131_확률보정.csv", index=False)

## 논문 feature 확인 --> train label 별 데이터 위주로

In [6]:
## Perplexity 확인

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Perplexity 계산 함수
def calculate_perplexity(text, model, tokenizer, device="cpu"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    input_ids = inputs["input_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        ppl = torch.exp(loss).item()
    return ppl

label_ppl = {}

human_ppls = []
for text in tqdm(label_0, desc=f"Label 0"):
    ppl = calculate_perplexity(text["full_text"], model, tokenizer, device=model.device)
    human_ppls.append(ppl)
label_ppl["Human"] = sum(human_ppls) / len(human_ppls) if human_ppls else float('inf')

AI_ppls = []
for text in tqdm(label_1, desc=f"Label 1"):
    ppl = calculate_perplexity(text["full_text"], model, tokenizer, device=model.device)
    AI_ppls.append(ppl)
label_ppl["AI"] = sum(AI_ppls) / len(AI_ppls) if AI_ppls else float('inf')

# 결과 출력
for label, ppl in label_ppl.items():
    print(f"Label: {label} | Avg PPL: {ppl:.2f}")

2025-07-03 13:13:24.533411: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 13:13:24.541344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751516004.549829  200487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751516004.552057  200487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 13:13:24.560346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Label: Human | Avg PPL: 7.96
Label: AI | Avg PPL: 7.66


In [ ]:
## subjectivity and objectivity
## 0에 가까울수록 객관적
## 1에 가까울수록 주관적

def get_subjectivity_score(text):
    if not isinstance(text, str):
        return 0.0
    return TextBlob(text).sentiment.subjectivity

# label_0 처리
label_0_df = pd.DataFrame(label_0)
label_0_df["label"] = 0
label_0_df["subjectivity"] = label_0_df["full_text"].apply(get_subjectivity_score)

# label_1 처리
label_1_df = pd.DataFrame(label_1)
label_1_df["label"] = 1
label_1_df["subjectivity"] = label_1_df["full_text"].apply(get_subjectivity_score)

# 합치기
combined_df = pd.concat([label_0_df, label_1_df], ignore_index=True)

# label별 평균 subjectivity 계산
label_subjectivity = combined_df.groupby("label")["subjectivity"].mean().to_dict()

# 출력
for label, score in label_subjectivity.items():
    print(f"Label: {label} | Avg Subjectivity: {score:.3f}")

Label: 0 | Avg Subjectivity: 0.033
Label: 1 | Avg Subjectivity: 0.027


In [ ]:
## stop word
# tqdm 설정
tqdm.pandas()

# 불용어 리스트 불러오기
stop_words_list = stopwords.words('/home/aicontest/nltk_data/corpora/stopwords/korean')
print('불용어 개수 :', len(stop_words_list))

# 형태소 분석기
okt = Okt()

# 불용어 개수 세기 함수
def count_korean_stopwords(text):
    if not isinstance(text, str):
        return 0
    tokens = okt.morphs(text)
    return sum(1 for token in tokens if token in stop_words_list)

# tqdm 적용한 불용어 개수 세기
label_0_df['stopword_count'] = label_0_df['full_text'].progress_apply(count_korean_stopwords)
label_1_df['stopword_count'] = label_1_df['full_text'].progress_apply(count_korean_stopwords)

# 결과 출력
print(f"불용어 개수 label 0 : {sum(label_0_df['stopword_count'])}")
print(f"불용어 개수 label 1 : {sum(label_1_df['stopword_count'])}")

불용어 개수 : 675


100%|██████████| 7995/7995 [08:21<00:00, 15.93it/s]

불용어 개수 label 0 : 19180068
불용어 개수 label 1 : 1671692


In [23]:
print(f"불용어 평균 label 0 : {sum(label_0_df['stopword_count'])/len(label_0_df)}")
print(f"불용어 평균 label 1 : {sum(label_1_df['stopword_count'])/len(label_1_df)}")

불용어 평균 label 0 : 215.07864135371227
불용어 평균 label 1 : 209.09218261413383


In [24]:
## quotation 측정

def count_quotation_marks(text):
    if not isinstance(text, str):
        return 0

    # 다양한 종류의 인용부호 포함
    quotation_marks = [
        '"', "'",         # 일반 쌍따옴표, 홑따옴표
        '“', '”',         # 유니코드 쌍따옴표
        '‘', '’',         # 유니코드 홑따옴표
        '„', '‟', '‹', '›', '«', '»'  # 기타 유럽권 인용부호
    ]

    return sum(text.count(mark) for mark in quotation_marks)

tqdm.pandas()

# 컬럼에 적용
label_0_df['quotation_count'] = label_0_df['full_text'].progress_apply(count_quotation_marks)
label_1_df['quotation_count'] = label_1_df['full_text'].progress_apply(count_quotation_marks)

# 요약 출력
print(f"Label 0 total quotations: {label_0_df['quotation_count'].sum()}")
print(f"Label 1 total quotations: {label_1_df['quotation_count'].sum()}")
print(f"Label 0 avg quotations: {label_0_df['quotation_count'].sum()/len(label_0_df)}")
print(f"Label 1 avg quotations: {label_1_df['quotation_count'].sum()/len(label_1_df)}")

100%|██████████| 7995/7995 [00:00<00:00, 140156.07it/s]

Label 0 total quotations: 701986
Label 1 total quotations: 56898
Label 0 avg quotations: 7.87182793769694
Label 1 avg quotations: 7.116697936210131


In [29]:
## 맞춤법

def count_spelling_errors(text):
    try:
        result = spell_checker.check(text)
        return result.errors  # 맞춤법 오류 개수 반환
    except Exception as e:
        print(f"Error processing text: {text[:30]}... -> {e}")
        return 0
    
tqdm.pandas()

label_0_df['spelling_error_count'] = label_0_df['full_text'].progress_apply(count_spelling_errors)
label_1_df['spelling_error_count'] = label_1_df['full_text'].progress_apply(count_spelling_errors)

print(f"Label 0 맞춤법 오류 총합: {label_0_df['spelling_error_count'].sum()}")
print(f"Label 1 맞춤법 오류 총합: {label_1_df['spelling_error_count'].sum()}")
print(f"Label 0 맞춤법 오류 평균: {label_0_df['spelling_error_count'].sum()/len(label_0_df)}")
print(f"Label 1 맞춤법 오류 평균: {label_1_df['spelling_error_count'].sum()/len(label_1_df)}")


100%|██████████| 7995/7995 [00:00<00:00, 77191.34it/s]

Error processing text: 그는 왕족이었고, 특히 그의 친가는 겹사돈으로 유명한 ... -> 'result'
Error processing text: 일본 하리마 국(播磨国) 출신이다.
쇼샤 산에서 천태교... -> 'result'
Error processing text: 리본은 히트맨이자 주인공인 츠나의 가정교사다. 그는 검... -> 'result'
Label 0 맞춤법 오류 총합: 0
Label 1 맞춤법 오류 총합: 0
Label 0 맞춤법 오류 평균: 0.0
Label 1 맞춤법 오류 평균: 0.0


In [2]:
## readability
label_0_df = pd.read_csv("./label_0_EDA.csv")
label_1_df = pd.read_csv("./label_1_EDA.csv")

okt = Okt()

def get_korean_readability(text):
    if not isinstance(text, str) or not text.strip():
        return {
            "avg_sentence_length": 0,
            "avg_word_length": 0,
            "num_sentences": 0,
            "num_words": 0,
        }

    # 문장 분리 (간단히 마침표 기준)
    sentences = re.split(r'[.!?。!?]', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    # 어절 분리
    words = text.split()
    morphs = okt.morphs(text)

    num_sentences = len(sentences)
    num_words = len(words)
    num_morphs = len(morphs)
    total_chars = sum(len(s) for s in sentences)

    return {
        "avg_sentence_length": total_chars / num_sentences if num_sentences else 0,
        "avg_word_length": total_chars / num_words if num_words else 0,
        "num_sentences": num_sentences,
        "num_words": num_words,
        "num_morphs": num_morphs
    }


tqdm.pandas()

readability_results = label_0_df['full_text'].progress_apply(get_korean_readability)
readability_df = pd.DataFrame(readability_results.tolist())

# 결과 합치기
label_0_df = pd.concat([label_0_df, readability_df], axis=1)

readability_results = label_1_df['full_text'].progress_apply(get_korean_readability)
readability_df = pd.DataFrame(readability_results.tolist())

# 결과 합치기
label_1_df = pd.concat([label_1_df, readability_df], axis=1)

100%|██████████| 7995/7995 [05:05<00:00, 26.21it/s]


In [3]:
# 측정할 컬럼 목록
readability_columns = [
    "avg_sentence_length",
    "avg_word_length",
    "num_sentences",
    "num_words",
    "num_morphs"
]

print("📊 평균 가독성 지표 (label_0_df):")
for col in readability_columns:
    mean_val = label_0_df[col].mean()
    print(f"{col:<20}: {mean_val:.2f}")

# 측정할 컬럼 목록
readability_columns = [
    "avg_sentence_length",
    "avg_word_length",
    "num_sentences",
    "num_words",
    "num_morphs"
]

print("📊 평균 가독성 지표 (label_1_df):")
for col in readability_columns:
    mean_val = label_1_df[col].mean()
    print(f"{col:<20}: {mean_val:.2f}")


📊 평균 가독성 지표 (label_0_df):
avg_sentence_length : 65.35
avg_word_length     : 4.41
num_sentences       : 36.05
num_words           : 511.16
num_morphs          : 959.69
📊 평균 가독성 지표 (label_1_df):
avg_sentence_length : 55.69
avg_word_length     : 4.33
num_sentences       : 40.89
num_words           : 515.54
num_morphs          : 949.96


In [11]:
tqdm.pandas()
okt = Okt()

# CSV 불러오기
label_0_df = pd.read_csv("/home/aicontest/aicon_2025/analysis/label_0_EDA.csv")
label_1_df = pd.read_csv("/home/aicontest/aicon_2025/analysis/label_1_EDA.csv")

# 1. 총 콤마 수
def count_commas(text):
    if not isinstance(text, str):
        return 0
    return text.count(',')

# 2. 콤마 포함 여부
def comma_inclusion(text):
    if not isinstance(text, str):
        return False
    return ',' in text

# 3. 콤마 사용 비율
def comma_usage_rate(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0
    return text.count(',') / len(text)

# 4. 콤마의 평균 상대 위치
def avg_relative_position_of_comma(text):
    if not isinstance(text, str) or ',' not in text:
        return 0
    positions = [i / len(text) for i, c in enumerate(text) if c == ',']
    return sum(positions) / len(positions) if positions else 0

# 5. 평균 세그먼트 길이 (콤마 기준)
def avg_segment_length(text):
    if not isinstance(text, str):
        return 0
    segments = [s.strip() for s in text.split(',')]
    lengths = [len(s) for s in segments if s]
    return sum(lengths) / len(lengths) if lengths else 0

# 6. 콤마 앞뒤 품사 다양성 점수
def pos_diversity_score(text):
    if not isinstance(text, str) or ',' not in text:
        return 0

    segments = [s.strip() for s in text.split(',') if s.strip()]
    if len(segments) < 2:
        return 0

    diversity_scores = []
    for i in range(len(segments) - 1):
        pos_1 = set(pos for _, pos in okt.pos(segments[i]))
        pos_2 = set(pos for _, pos in okt.pos(segments[i + 1]))

        union = pos_1.union(pos_2)
        intersection = pos_1.intersection(pos_2)

        score = 1 - len(intersection) / len(union) if union else 0
        diversity_scores.append(score)

    return sum(diversity_scores) / len(diversity_scores) if diversity_scores else 0


# 모든 feature를 DataFrame에 적용
def apply_features(df):
    df["comma_count"] = df["full_text"].progress_apply(count_commas)
    df["comma_inclusion"] = df["full_text"].progress_apply(comma_inclusion)
    df["comma_usage_rate"] = df["full_text"].progress_apply(comma_usage_rate)
    df["relative_comma_pos"] = df["full_text"].progress_apply(avg_relative_position_of_comma)
    df["avg_segment_length"] = df["full_text"].progress_apply(avg_segment_length)
    df["pos_diversity_score"] = df["full_text"].progress_apply(pos_diversity_score)
    return df

label_0_df = apply_features(label_0_df)
label_1_df = apply_features(label_1_df)

# 요약 출력
print("=== Summary ===")
print(f"Label 0 total commas: {label_0_df['comma_count'].sum()}")
print(f"Label 1 total commas: {label_1_df['comma_count'].sum()}")

print(f"Label 0 avg commas: {label_0_df['comma_count'].mean():.4f}")
print(f"Label 1 avg commas: {label_1_df['comma_count'].mean():.4f}")

print(f"Label 0 comma inclusion rate: {label_0_df['comma_inclusion'].mean():.4f}")
print(f"Label 1 comma inclusion rate: {label_1_df['comma_inclusion'].mean():.4f}")

for metric in ["comma_usage_rate", "relative_comma_pos", "avg_segment_length", "pos_diversity_score"]:
    print(f"Label 0 avg {metric}: {label_0_df[metric].mean():.4f}")
    print(f"Label 1 avg {metric}: {label_1_df[metric].mean():.4f}")

100%|██████████| 7995/7995 [12:03<00:00, 11.05it/s]

=== Summary ===
Label 0 total commas: 2225240
Label 1 total commas: 132245
Label 0 avg commas: 24.9531
Label 1 avg commas: 16.5410
Label 0 comma inclusion rate: 0.9950
Label 1 comma inclusion rate: 0.9800
Label 0 avg comma_usage_rate: 0.0111
Label 1 avg comma_usage_rate: 0.0072
Label 0 avg relative_comma_pos: 0.4789
Label 1 avg relative_comma_pos: 0.4742
Label 0 avg avg_segment_length: 114.5820
Label 1 avg avg_segment_length: 186.9172
Label 0 avg pos_diversity_score: 0.4479
Label 1 avg pos_diversity_score: 0.4446


In [ ]:
label_0_df.to_csv("./label_0_EDA.csv")
label_1_df.to_csv("./label_1_EDA.csv")